In [1]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import sys
import os

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [2]:
import numpy as np
from src.rocket import Rocket

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

# Rocket setup
Ts = 1 / 20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)


print("Linearization around a steady state:")

xs, us = rocket.trim()
print("Default trim function:")
print("xs = ", xs)
print("us = ", us)

xs, us = rocket.trim()
print("xs = ", xs)
print("us = ", us)

x_ref = np.array([0.0] * 9 + [1.0, 0.0, 3.0])
xs, us = rocket.trim(x_ref)
print("x_ref = ", x_ref)
print("xs = ", xs)
print("us = ", us)

Linearization around a steady state:

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Default trim function:
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
us =  [ 0.          0.         66.66666667  0.        ]
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
us =  [ 0.          0.         66.66666667  0.        ]
x_ref =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
us =  [ 0.          0.         66.66666667  0.        ]


In [3]:
sys = rocket.linearize_sys(xs, us)
sys.info()  # see whole system


 System name: 'complete system'
------------------------------------------------------------
self.A shape: (12, 12)
self.B shape: (12, 4)
self.C shape: (12, 12)
self.D shape: (12, 4)
self.xs: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
self.us: [ 0.          0.         66.66666667  0.        ]

🔹 State variables:
        wx [rad/s]
        wy [rad/s]
        wz [rad/s]
     alpha [rad]
      beta [rad]
     gamma [rad]
        vx [m/s]
        vy [m/s]
        vz [m/s]
         x [m]
         y [m]
         z [m]
→ access via self.state_name, self.state_unit

🔹 Input variables:
        d1 [rad]
        d2 [rad]
      Pavg [%]
     Pdiff [%]
→ access via self.input_name, self.input_unit

🔹 Output variables:
        wx [rad/s]
        wy [rad/s]
        wz [rad/s]
     alpha [rad]
      beta [rad]
     gamma [rad]
        vx [m/s]
        vy [m/s]
        vz [m/s]
         x [m]
         y [m]
         z [m]
→ access via self.output_name, self.output_unit
-------------------------------------

In [12]:
sys_x, sys_y, sys_z, sys_roll = sys.decompose()
sys_x.info()


 System name: 'sys_x'
------------------------------------------------------------
self.A shape: (4, 4)
self.B shape: (4, 1)
self.C shape: (1, 4)
self.D shape: (1, 1)
self.xs: [0. 0. 0. 1.]
self.us: [0.]
self.idx: [1 4 6 9]
self.idu: [1]
self.idy: [9]

🔹 State variables:
        wy [rad/s]
      beta [rad]
        vx [m/s]
         x [m]
→ access via self.state_name, self.state_unit

🔹 Input variables:
        d2 [rad]
→ access via self.input_name, self.input_unit

🔹 Output variables:
         x [m]
→ access via self.output_name, self.output_unit
------------------------------------------------------------
 To check data directly, try:
   sys.A, sys.B, sys.C, sys.D
   sys.xs, sys.us, sys.state_name, etc.
------------------------------------------------------------


In [11]:
print("Ax",sys_x.A, "Bx",sys_x.B, "Cx",sys_x.C, "Dx",sys_x.D)
print(sys_y.A, sys_y.B, sys_y.C, sys_y.D)
print(sys_z.A, sys_z.B, sys_z.C, sys_z.D)
print(sys_roll.A, sys_roll.B, sys_roll.C, sys_roll.D)

Ax [[0.    0.    0.    0.   ]
 [1.    0.    0.    0.   ]
 [0.    9.806 0.    0.   ]
 [0.    0.    1.    0.   ]] Bx [[-65.47484472]
 [  0.        ]
 [  9.806     ]
 [  0.        ]] Cx [[0. 0. 0. 1.]] Dx [[0.]]
[[ 0.     0.     0.     0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.    -9.806  0.     0.   ]
 [ 0.     0.     1.     0.   ]] [[-65.47484472]
 [  0.        ]
 [ -9.806     ]
 [  0.        ]] [[0. 0. 0. 1.]] [[0.]]
[[0. 0.]
 [1. 0.]] [[0.14709]
 [0.     ]] [[0. 1.]] [[0.]]
[[0. 0.]
 [1. 0.]] [[0.104]
 [0.   ]] [[0. 1.]] [[0.]]


In [ ]:
Ad, Bd, Cd = sys_x._discretize(Ts)
print(Ad, Bd, Cd)

In [ ]:
[Ad, Bd, Cd] = sys_y._discretize(Ts)
Ad, Bd, Cd

In [ ]:
[Ad, Bd, Cd] = sys_z._discretize(Ts)
Ad, Bd, Cd

In [ ]:
[Ad, Bd, Cd] = sys_roll._discretize(Ts)
Ad, Bd, Cd